In [1]:
# ideas for a source test
import cobra
import pandas as pd

from refinegems.utility.io import load_model

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [13]:
test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = refinegems.utility.io.load_model(test_model_path,'cobra')


AttributeError: module 'refinegems.utility.io' has no attribute 'load_model'

-----
### Plot the basic analysis

- one model
- collection of models

In [ ]:
# old version of how to plot basic analysis
# will absolutely no longer work since it uses a model and the report is no longer based on it.
def plot_initial_analysis(models: list[libModel]):
    """Creates bar plot of number of entities per Model

    Args:
        - models (list[libModel]): Models loaded with libSBML

    Returns:
        plot: Pandas Barchart
    """
    numbers = pd.DataFrame([initial_analysis(model) for model in models], columns=['model', 'metabolites', 'reactions', 'genes'])
    ax = numbers.set_index('model').plot.bar(y=['metabolites', 'reactions', 'genes'], figsize=(8, 5), cmap='Paired', rot=0)
    # commented is possibility to integrate memote scores
    #numbers.set_index('model').plot(y='Memote score', ax=ax, use_index=False, linestyle=':', secondary_y='Memote score', color='k', marker='D', legend=True)
    #ax.right_ax.set_ylabel('Memote score [%]')
    #ax.right_ax.legend(loc='upper right', bbox_to_anchor=[0.98, 0.9])
    #ax.right_ax.set_ylim([75, 95])
    ax.legend(title=False, loc='upper left', ncol=3, frameon=False)
    ylim = numbers.drop('model', axis=1).max().max() + 200
    ax.set_ylim([0,ylim])
    ax.set_xlabel('')
    ax.tick_params(axis='x',which='both', bottom=False,top=False)
    return ax

In [ ]:
from refinegems.classes.reports import ModelInfoReport

rep = ModelInfoReport(test_model)